In [ ]:
import numpy as np
import random

In [ ]:
def forward_sim(n,T, alpha=0.9, gamma=0.1, beta=0.2, lmda=(1, 5)): 
    Gamma = [[1-gamma, 0, gamma], [0, 1-gamma, gamma], [beta*0.5, beta*0.5, 1-beta]]
    Z_prob = [[alpha, 1-alpha], [1-alpha, alpha], [0.5, 0.5]]
    C = np.zeros(T,dtype=int)
    Z = np.zeros((n,T), dtype=int)
    X = np.zeros((n,T), dtype=int)
    C[0] = 2
    print(C[0])
    for i in range(0,n):
            Z[i,0] = random.choices([0,1], Z_prob[C[0]], k = 1)[0]
            X[i][0] = np.random.poisson(lam = lmda[Z[i,0]])

    for t in range(1,T):
        C[t] = random.choices([0,1,2], Gamma[C[t-1]], k = 1)[0]
        for i in range(0,n):
            Z[i][t] = random.choices([0,1], Z_prob[C[t]], k = 1)[0]
            X[i][t] = np.random.poisson(lam = lmda[Z[i,t]])
    return C, Z, X



     



In [ ]:
C, Z, X = forward_sim(10,1000)

In [ ]:
C

In [ ]:
Z

In [ ]:
X